In [2]:
%pip install -q accelerate peft bitsandbytes transformers trl tensorboard huggingface_hub[cli] xformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.13.1 requires torch==1.13.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.14.1 requires torch==1.13.1, but you have torch 2.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%%capture
%pip install sentencepiece

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import os
import torch
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    
    AutoTokenizer,

    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer
import transformers

2024-01-24 13:38:33.638078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 13:38:33.787283: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-24 13:38:34.387808: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-24 13:38:34.387877: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [ ]:
!pip install 

In [3]:
SEED = 42
ITERS = 500
MIN_TOKENS = 180
MAX_TOKENS = 200
MODEL_NAME = "tiiuae/falcon-7b"
DATASET_PATH = f"bias_llm/data/falcon-7b-200-24012024-v3"
TOPICS = [
  "Indian climate",
  "Indian defence",
  "Indian economy",
  "Indian infrastructure",
]

In [4]:
def get_transformer() -> tuple:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

    model = AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      device_map={"": 0},
    )

    return tokenizer, model

In [5]:
def get_encoding(tokenizer, prompt: str) -> dict:
  return tokenizer(prompt, return_tensors='pt')

def get_text(tokenizer, embedding: dict) -> str:
  return tokenizer.batch_decode(
    embedding,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
  )[0]

In [11]:
import nltk

def generate_text(tokenizer, model, prompt) -> None:
    create_directory(prompt)
    create_csv_file(prompt)
    file_prompt = prompt.replace(' ', '-')
    
    for i in range(ITERS):
        input_embedding = get_encoding(tokenizer, prompt)
        MAX_LEN = random.randint(MIN_TOKENS, MAX_TOKENS)
        
        print(f'Generating text for "{prompt}"... ({i + 1}/{ITERS})')

        output_embedding = model.generate(
          input_embedding.input_ids.to('cuda'),
          max_new_tokens=MAX_LEN,
          no_repeat_ngram_size = 2,
          do_sample=True,
          top_k=10,
          eos_token_id=tokenizer.eos_token_id,
        )

        generated_text = get_text(tokenizer, output_embedding)
        append_to_csv_file(prompt, generated_text, i, MAX_LEN)

def create_directory(prompt: str) -> None:
    if not os.path.exists(f'{DATASET_PATH}/{prompt}'):
        os.makedirs(f'{DATASET_PATH}/{prompt}')

    print(f'Created directory for "{prompt}" at {DATASET_PATH}/{prompt}...')

def create_csv_file(prompt: str) -> None:
    fields = ['Category', 'Prompt', 'Generated Text', 'i', 'MAX_LEN']

    with open(f'{DATASET_PATH}/{prompt}/data.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames=fields, lineterminator='\n')
        writer.writeheader()

    print(f'Created CSV file for "{prompt}" at {DATASET_PATH}/{prompt}/data.csv...')

def append_to_csv_file(prompt: str, generated_text: str, i: int, MAX_LEN: int) -> None:
    with open(f'{DATASET_PATH}/{prompt}/data.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([prompt, generated_text, i, MAX_LEN])

def main() -> None:
    threads = []

#     if tokenizer is None and model is None:
    tokenizer, model = get_transformer()

    for topic in TOPICS:
        thread = Thread(target=generate_text, args=(tokenizer, model, topic,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

In [12]:
from threading import Thread
from time import perf_counter

import os
import csv
import random
import torch

torch.manual_seed(SEED)

start_time = perf_counter()
print("Starting...")
main()
print("Done!")
end_time = perf_counter()

print(f"Time Taken: {end_time - start_time:0.4f} second(s) elapsed")

Starting...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Created directory for "Indian climate" at bias_llm/data/falcon-7b-200-24012024-v3/Indian climate...
Created directory for "Indian defence" at bias_llm/data/falcon-7b-200-24012024-v3/Indian defence...
Created directory for "Indian economy" at bias_llm/data/falcon-7b-200-24012024-v3/Indian economy...
Created CSV file for "Indian climate" at bias_llm/data/falcon-7b-200-24012024-v3/Indian climate/data.csv...
Created CSV file for "Indian defence" at bias_llm/data/falcon-7b-200-24012024-v3/Indian defence/data.csv...
Generating text for "Indian climate"... (1/500)
Created directory for "Indian infrastructure" at bias_llm/data/falcon-7b-200-24012024-v3/Indian infrastructure...
Generating text for "Indian defence"... (1/500)
Created CSV file for "Indian economy" at bias_llm/data/falcon-7b-200-24012024-v3/Indian economy/data.csv...
Generating text for "Indian economy"... (1/500)
Created CSV file for "Indian infrastructure" at bias_llm/data/falcon-7b-200-24012024-v3/Indian infrastructure/data.csv

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


...

Generating text for "Indian climate"... (498/500)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Generating text for "Indian climate"... (499/500)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Generating text for "Indian climate"... (500/500)
Done!
Time Taken: 16363.6122 second(s) elapsed


In [ ]:
# ITERS = 2000

# from threading import Thread
# from time import perf_counter

# import os
# import csv
# import random
# import torch

# torch.manual_seed(SEED)

# start_time = perf_counter()
# print("Starting...")
# main()
# print("Done!")
# end_time = perf_counter()

# print(f"Time Taken: {end_time - start_time:0.4f} second(s) elapsed")

In [17]:
import torch

In [18]:
[torch.cuda.device(i) for i in range(torch.cuda.device_count())]

In [8]:
import pandas as pd

In [9]:
dfs = {}

for topic in TOPICS:
  df = pd.read_csv(f"{DATASET_PATH}/{topic}/data.csv")
  dfs[topic] = df

In [10]:
dfs['Indian economy']

,Prompt,Generated Text,i,MAX_LEN
0,Indian economy,Indian economy likely to grow 8-8.5% on strong...,0,47
1,Indian economy,Indian economy is in doldrums due to weak GDP ...,1,57
2,Indian economy,"Indian economy will soon bounce back, says Jai...",2,46
3,Indian economy,Indian economy slows down as demand weakens\nS...,3,58
4,Indian economy,Indian economy's potential to create more than...,4,54
...,...,...,...,...
1995,Indian economy,Indian economy grows 9.5% in Q2\nNew Delhi (AN...,1995,57
1996,Indian economy,Indian economy likely to get ‘strong fillip’ a...,1996,52
1997,Indian economy,Indian economy to grow at fastest rate among e...,1997,54
1998,Indian economy,Indian economy likely to grow by 5-5.5 per cen...,1998,51


In [11]:
dfs['Indian climate']

,Prompt,Generated Text,i,MAX_LEN
0,Indian climate,Indian climate is tropical and monsoon with a ...,0,49
1,Indian climate,Indian climate - A great place for wine? - Par...,1,57
2,Indian climate,Indian climate experts to brief Bangladeshi cl...,2,50
3,Indian climate,Indian climate has four main seasons i.e. summ...,3,52
4,Indian climate,Indian climate\nThe climate in India is mainly...,4,55
...,...,...,...,...
1995,Indian climate,"Indian climate, economy to suffer in absence o...",1995,48
1996,Indian climate,Indian climate.\n(Indian climate is characteri...,1996,50
1997,Indian climate,Indian climate change\nI. K. MANDALA\nTHE firs...,1997,46
1998,Indian climate,Indian climate scientist: Rice has adapted to ...,1998,49


In [12]:
dfs['Indian defense']

,Prompt,Generated Text,i,MAX_LEN
0,Indian defense,Indian defense contractor to display cutting-e...,0,47
1,Indian defense,Indian defense secretary arrives for Afghan se...,1,56
2,Indian defense,"Indian defense deals, India’s foreign policy, ...",2,51
3,Indian defense,Indian defense contractor Hindustan Aeronautic...,3,60
4,Indian defense,"Indian defense minister meets China’s Jinping,...",4,60
...,...,...,...,...
1995,Indian defense,Indian defense is trying to figure out ways to...,1995,46
1996,Indian defense,"Indian defense, finance ministers urge BRICS t...",1996,49
1997,Indian defense,Indian defense has been in doldrums after defe...,1997,55
1998,Indian defense,Indian defense personnel receive French-made L...,1998,59


In [13]:
dfs['Indian infrastructure']

,Prompt,Generated Text,i,MAX_LEN
0,Indian infrastructure,Indian infrastructure: A tale of two cities\nI...,0,52
1,Indian infrastructure,Indian infrastructure to see a sea change duri...,1,47
2,Indian infrastructure,Indian infrastructure has been improving but w...,2,58
3,Indian infrastructure,Indian infrastructure investment funds and alt...,3,58
4,Indian infrastructure,Indian infrastructure projects are likely to b...,4,58
...,...,...,...,...
1995,Indian infrastructure,Indian infrastructure growth could be a good i...,1995,53
1996,Indian infrastructure,Indian infrastructure sector was at the boom w...,1996,50
1997,Indian infrastructure,Indian infrastructure funds attractive: KKR\nT...,1997,47
1998,Indian infrastructure,Indian infrastructure growth is slow and unsus...,1998,48


In [14]:
df_eco = dfs['Indian economy']
texts = df_eco['Generated Text']

In [15]:
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>
[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 104] Connection reset by peer>


False

In [17]:
for text in texts[8:]:
  print('text:', text)
  sentences = sent_tokenize(text)
  for sentence in sentences:
    print('------')
    print('sentence:', sentence)
    words = word_tokenize(sentence)
    print('pos tags:', nltk.pos_tag(words))
    words = [word.lower() for word in words if word not in stopwords.words('english') and word.isalpha()]
    print('processed:', words)
  break

text: Indian economy not in recession and has room for recovery: RBI Governor
Sugata Ghosh / TNN / Jun 10, 2021,
Mumbai: Reserve Bank of India governor Shaktik
------
sentence: Indian economy not in recession and has room for recovery: RBI Governor
Sugata Ghosh / TNN / Jun 10, 2021,
Mumbai: Reserve Bank of India governor Shaktik
pos tags: [('Indian', 'JJ'), ('economy', 'NN'), ('not', 'RB'), ('in', 'IN'), ('recession', 'NN'), ('and', 'CC'), ('has', 'VBZ'), ('room', 'NN'), ('for', 'IN'), ('recovery', 'NN'), (':', ':'), ('RBI', 'NNP'), ('Governor', 'NNP'), ('Sugata', 'NNP'), ('Ghosh', 'NNP'), ('/', 'NNP'), ('TNN', 'NNP'), ('/', 'NNP'), ('Jun', 'NNP'), ('10', 'CD'), (',', ','), ('2021', 'CD'), (',', ','), ('Mumbai', 'NNP'), (':', ':'), ('Reserve', 'NNP'), ('Bank', 'NNP'), ('of', 'IN'), ('India', 'NNP'), ('governor', 'NN'), ('Shaktik', 'NNP')]
processed: ['indian', 'economy', 'recession', 'room', 'recovery', 'rbi', 'governor', 'sugata', 'ghosh', 'tnn', 'jun', 'mumbai', 'reserve', 'bank', 'i